In [1]:

!pip install git+https://github.com/ultralytics/ultralytics.git@main


  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-0jvcu_ht
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-0jvcu_ht
  Resolved https://github.com/ultralytics/ultralytics.git to commit 95a9796fee5a110fdd861e12b476b1dfa237b910
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.44-py3-none-any.whl size=898706 sha256=088dadd7728d5a77d88cec6d250d216b30b42de27fd9d290cbf51f191275bbf6
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrk0r3hf/wheels/60/db/e5/6abbdd13b4e3d5e2fdcd87690cd8738f842eeb37142fdccb5b
Successfully built ultralytics


In [2]:
import os
import shutil
import sys
import json
import base64
import cv2
import numpy as np
import io

In [3]:
def recopy_to_dir(current_dir, target_dir):
  if not os.path.exists(target_dir):
    os.mkdir(target_dir)
  image_names = os.listdir(current_dir)
  for name in image_names:
    shutil.copy(os.path.join(current_dir,name), target_dir)



In [5]:
recopy_to_dir("/content/drive/MyDrive/nornickel2/cv_open_dataset/open_img", "/content/sample_data/dataset/images/train")
recopy_to_dir("/content/drive/MyDrive/nornickel2/cv_synt_dataset/synt_img","/content/sample_data/dataset/images/train")

In [6]:
def convert_mask_to_yolo(mask_image):

    if mask_image is None:
        print("Не удалось обработать маску")
        return None


    black_mask = cv2.inRange(mask_image, (0, 0, 0), (50, 50, 50))


    new_image = np.ones_like(mask_image) * 255
    new_image[black_mask > 0] = [0, 0, 0]

    return new_image


In [7]:
from PIL import Image, ImageDraw
from IPython.display import display

In [8]:
def create_mask(mask_dir):
  mask = cv2.imread(mask_dir, cv2.IMREAD_GRAYSCALE)
  binary_mask = convert_mask_to_yolo(cv2.imread(mask_dir))
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  objects_info = []

  contour_image = np.zeros_like(mask)
  bounding_rect_image = mask.copy()
  yolo_bbox_image = mask.copy()
  for contour in contours:
      x, y, width, height = cv2.boundingRect(contour)
      class_label = 0
      x_center, y_center, normalized_width, normalized_height = convert_coordinates_to_yolo(mask.shape[1], mask.shape[0], x, y, width, height)
      objects_info.append((class_label, x_center, y_center, normalized_width, normalized_height))

      cv2.rectangle(bounding_rect_image, (x, y), (x + width, y + height), 255, thickness=2)

      cv2.drawContours(contour_image, [contour], 0, 255, thickness=2)

  return objects_info

def convert_coordinates_to_yolo(image_width, image_height, x, y, width, height):
    x_center = (x + width / 2) / image_width
    y_center = (y + height / 2) / image_height
    normalized_width = width / image_width
    normalized_height = height / image_height

    return x_center, y_center, normalized_width, normalized_height
    cv2.imwrite(os.path.join(output_dir,name), arr)

def write_yolo_annotations(output_path, image_name, objects_info):
    annotation_file_path = os.path.join(output_path, image_name)

    with open(annotation_file_path, "w") as file:
        for obj_info in objects_info:
            line = f"{obj_info[0]} {obj_info[1]} {obj_info[2]} {obj_info[3]} {obj_info[4]}\n"
            file.write(line)

In [ ]:
'''input_path = "/content/drive/MyDrive/nornickel/cv_open_dataset/open_msk"
output_path = "/content/sample_data/test5"

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Trying it on one mask
mask_path = '/content/drive/MyDrive/nornickel/cv_open_dataset/open_msk/1710277054_0.png'
image_name = os.path.basename(mask_path).replace(".png", ".txt")

objects_info = create_mask(mask_path)
print(objects_info)
write_yolo_annotations(output_path, image_name, objects_info)'''

[(0, 0.40989583333333335, 0.8495370370370371, 0.23854166666666668, 0.30092592592592593), (0, 0.1625, 0.07314814814814814, 0.203125, 0.14629629629629629)]


In [9]:
d = {}
d['train'] = "/content/sample_data/dataset/images/train"
d['nc'] = 1
d['names'] = ['class_1']
d['val'] = "/content/sample_data/dataset/images/val"

In [10]:
import yaml

In [11]:
with open('/content/sample_data/dataset/data.yaml', 'w') as outfile:
    yaml.dump(d, outfile, default_flow_style=False)


In [12]:
def write_masks(mask_dir, output_path):
  os.makedirs(output_path, exist_ok=True)
  names = os.listdir(mask_dir)
  for name in names:
    mask_path = os.path.join(mask_dir, name)
    image_name = os.path.basename(mask_path).replace(".png", ".txt")
    objects_info = create_mask(mask_path)
    write_yolo_annotations(output_path, image_name, objects_info)

In [13]:
write_masks("/content/drive/MyDrive/nornickel2/cv_open_dataset/open_msk/", "/content/sample_data/dataset/labels/train")
write_masks("/content/drive/MyDrive/nornickel2/cv_synt_dataset/synt_msk", "/content/sample_data/dataset/labels/train")

In [14]:
def train_val_split(images_train_dir, labels_train_dir, images_val, labels_val, train_size=0.7):
  imgs = sorted(os.listdir(images_train_dir))
  labels = sorted(os.listdir(labels_train_dir))
  indxs = np.random.permutation(len(imgs))
  size = int(len(indxs)*train_size)
  indxs = indxs[size:]
  for ind in indxs:
    shutil.move(os.path.join(images_train_dir,imgs[ind]), os.path.join(images_val,imgs[ind]))
    shutil.move(os.path.join(labels_train_dir,labels[ind]), os.path.join(labels_val,labels[ind]))

In [15]:
train_val_split("/content/sample_data/dataset/images/train", "/content/sample_data/dataset/labels/train", "/content/sample_data/dataset/images/val", "/content/sample_data/dataset/labels/val")

In [16]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [17]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 44.2MB/s]


In [18]:
import torch

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
 model.train(data="/content/sample_data/dataset/data.yaml", epochs=100, imgsz=840, device=device)

engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/sample_data/dataset/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=840, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=Fals

100%|██████████| 755k/755k [00:00<00:00, 10.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/sample_data/dataset/labels/train... 277 images, 120 backgrounds, 0 corrupt: 100%|██████████| 277/277 [00:00<00:00, 398.26it/s]

train: New cache created: /content/sample_data/dataset/labels/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/sample_data/dataset/labels/val... 120 images, 43 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<00:00, 465.35it/s]

val: New cache created: /content/sample_data/dataset/labels/val.cache


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 864 train, 864 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
metrics = model.val(save_json=True)